In [1]:
# PyTorch 中以下数据结构分为 CPU 和 GPU 两个版本
# Tensor、Variable(包括 Parameter)、nn.Module(包括常用的layer、loss function, 以及容器 Sequential等)

In [2]:
from torch import nn
import torch as t

In [3]:
t.__version__

'0.3.1'

In [4]:
tensor = t.Tensor(3, 4)
# 返回一个新的 tensor，保存在第1块GPU上，但原来的tensor并没有改变
tensor.cuda(0)
tensor.is_cuda #False

/home/mzl/virtualenv0.3/venv/lib/python3.6/site-packages/torch/cuda/__init__.py:97: UserWarning: 
    Found GPU0 GeForce GTX 950M which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


False

In [5]:
# 不指定所使用的 GPU 设备，将默认使用第一快 GPU
tensor = tensor.cuda()
tensor.is_cuda # True

True

In [6]:
variable = t.autograd.Variable(tensor)
variable.cuda()
variable.is_cuda # True

True

In [7]:
module = nn.Linear(3, 4)
module.cuda(device = 0)
module.weight.is_cuda # True

True

In [8]:
class VeryBigModule(nn.Module):
    def __init__(self):
        super(VeryBigModule, self).__init__()
        self.GiantParameter1 = t.nn.Parameter(t.randn(100000, 20000)).cuda(0)
        self.GiantParameter2 = t.nn.Parameter(t.randn(20000, 100000)).cuda(1)
        
    def forward(self, x):
        x = self.GiantParameter1.mm(x.cuda(0))
# 两个 Parameter 所占的内存空间都非常大，大概是 8GB，如果将这两个 Parameter 同时放在一个 GPU 上几乎会
# 将显存占满，无法再进行任何其他运算。此时可以通过这种方式将不同的计算分布到不同的 GPU 中
#        x = self.GiantParameter2.mm(x.cuda(1))
        return x

In [9]:
# 关于使用 GPU 一些额建议：
# 1. GPU 运算速度快，但运算量小，并不能体现出它的优势，因此一些简单的操作可以直接利用CPU完成
# 2. 数据在 CPU 和 GPU 之间的传播会比较耗时，应当尽量避免
# 3. 在进行低精度的计算时，可以考虑 HalfTensor，相比 FloatTensor 能节省一半的显存
# 但需千万注意数值溢出的情况

In [10]:
# 注意：大部分的损失函数也属于 nn.Module

# 交叉熵损失函数，带权重
criterion = t.nn.CrossEntropyLoss(weight = t.Tensor([1, 3]))
input = t.autograd.Variable(t.randn(4, 2)).cuda()
target = t.autograd.Variable(t.Tensor([1, 0, 0, 1])).long().cuda()

#下面这行会报错，因为 weight 未被转移至 GPU
#loss = criterion(input, target)

#这行也会报错
criterion.cuda()
#loss = criterion(input, target)

criterion._buffers

OrderedDict([('weight', 
               1
               3
              [torch.cuda.FloatTensor of size 2 (GPU 0)])])

In [ ]:
# 注：pytorch 不支持型号过低的显卡
# 显卡如果过低，会报错
# 除了调用对象 .cuda 方法外，还可以使用 torch.cuda.device 指定默认使用哪一块 GPU
# 或使用 torch.set_default_tensor_type 使程序默认使用 GPU ，不需要手动调用 cuda
# 如果未指定使用哪一块 GPU，默认使用 GPU 0
x = t.cuda.FloatTensor(2, 3)
# x.get_device() == 0
y = t.FloatTensor(2, 3).cuda()
# y.get_device() == 0

#指定默认使用 GPU 0
with t.cuda.device(0):
    # 在 GPU 1 上构建 tensor
    a = t.cuda.FloatTensor(2, 3)
    
    # 将 tensor 转移至 GPU 1
    b = t.FloatTensor(2, 3).cuda()
    print('66',a.get_device() == b.get_device() == 0)
    
    c = a + b
    print('66',c.get_device() == 0)
    
    z = x + y
    print('66',z.get_device() == 0)
    
    # 手动指定使用 GPU 0
    d = t.randn(2, 3).cuda(0)
    
    print('66',d.get_device() == 2)

In [ ]:
t.set_default_tensor_type('torch.cuda.FloatTensor') 
# 指定默认 tensor 的类型为 GPU 上的 FloatTensor

In [ ]:
a = t.ones(2, 3)
a.is_cuda

In [ ]:
# 可以通过 t.save(obj, file_name) 等方法保存任意可序列化的对象
# 然后通过 obj = t.load(file_name) 方法加载保存的数据
a = t.Tensor(3, 4)
if t.cuda.is_available():
    a = a.cuda(0) # 把 a 转为 GPU1 上的tensor
    t.save(a, 'a.pth')
    
    # 加载为 b， 存储于 GPU1 上（因为保存时 tensor 就在 GPU1 上）
    b = t.load('a.pth')
    
    # 加载为 c， 存储于 CPU
    c = t.load('a.pth', map_location = lambda storage, loc: storage)
    
    # 加载为 d, 存储于 GPU0 上
    d = t.load('a.pth', map_location = {'cuda:1':'cuda:0'})

In [ ]:
t.set_default_tensor_type('torch.FloatTensor')
from torchvision.models import AlexNet
model = AlexNet()
# module 的 state_dict 是一个字典
model.state_dict().keys()

In [ ]:
# Module 对象的保存与加载
t.save(model.state_dict(), 'alexnet.pth')
model.load_state_dict(t.load('alexnet.pth'))

In [ ]:
optimizer = t.optim.Adam(model.parameters(), lr = 0.1)

In [ ]:
t.save(optimizer.state_dict(), 'optimizer.pth')
optimizer.load_state_dict(t.load('optimizer.pth'))

In [ ]:
all_data = dict(
    optimizer = optimizer.state_dict(),
    model = model.state_dict(),
    info = u'模型和优化器的所有参数'
)
t.save(all_data, 'all.pth')

In [ ]:
all_data = t.load('all.pth')
all_data.keys()